# Análise Gráfica e Resumo de Dados

## 1. Importação de Bibliotecas

In [3]:
import folium

## 2. Análise Gráfica

In [4]:
def criar_mapa_apartamentos(data):
    
    # criação do mapa centralizado pela média das coordenadas
    mapa = folium.Map(
        location=[data['latitude'].mean(), data['longitude'].mean()],
        zoom_start=12
    )
    
    # marcadores de círculo coloridos
    for _, row in data.iterrows():
        # definição de cor baseada no preço e número de reviews
        if row['price'] < 50 and row['numero_de_reviews'] > 100:
            cor = 'green'
        elif row['price'] < 100 and row['numero_de_reviews'] > 50:
            cor = 'yellow'
        elif row['price'] < 200:
            cor = 'orange'
        else:
            cor = 'red'
        
        # adição do marcador ao mapa
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=8,
            color=cor,
            fill=True,
            fill_color=cor,
            fill_opacity=0.6,
            popup=f"Bairro: {row['bairro']}, Preço: ${row['price']}, Reviews: {row['numero_de_reviews']}" # quando clicar em cima do circulo verá essas informações
        ).add_to(mapa)
    
    return mapa

## Análise de Bairros

In [7]:
def analisar_bairros(data):

    def definir_cor(row):
        if row['price'] < 50 and row['numero_de_reviews'] > 100:
            return 'green'
        elif row['price'] < 100 and row['numero_de_reviews'] > 50:
            return 'yellow'
        elif row['price'] < 200:
            return 'orange'
        else:
            return 'red'
    
    data['cor'] = data.apply(definir_cor, axis=1)
    
    cores = ['green', 'yellow', 'orange', 'red']
    
    resumo_bairros = data.groupby('bairro_original')['cor'].value_counts().unstack(fill_value=0)
    
    # adicionar colunas de cores ausentes se elas não existirem
    for cor in cores:
        if cor not in resumo_bairros.columns:
            resumo_bairros[cor] = 0
    
    resumo_percentual = resumo_bairros.div(resumo_bairros.sum(axis=1), axis=0) * 100
    
    # ordenação por soma de percentuais de green e yellow (bons investimentos)
    bairros_ordenados = resumo_percentual.assign(
        total_verde_amarelo=resumo_percentual['green'] + resumo_percentual['yellow']
    ).sort_values('total_verde_amarelo', ascending=False)
    
    # resumo na saída do notebook
    output_lines = []
    output_lines.append("Análise de Bairros para Aluguel (Ordenados por Green e Yellow):\n")
    
    for bairro in bairros_ordenados.index:
        linhas_bairro = [f"Bairro: {bairro}", f"Total de imóveis: {resumo_bairros.loc[bairro].sum()}"]
        for cor in cores:
            percentual = resumo_percentual.loc[bairro, cor]
            if percentual > 0:
                if cor == 'green':
                    linhas_bairro.append(f"Green (Ótimo para aluguel): {percentual:.2f}%")
                elif cor == 'yellow':
                    linhas_bairro.append(f"Yellow (Bom para aluguel): {percentual:.2f}%")
                elif cor == 'orange':
                    linhas_bairro.append(f"Orange (Intermediário): {percentual:.2f}%")
                elif cor == 'red':
                    linhas_bairro.append(f"Red (Menos recomendado): {percentual:.2f}%")
        linhas_bairro.append("\n")
        output_lines.append("\n".join(linhas_bairro))
    
    # imprimir as primeiras análises
    print("\n".join(output_lines[:5]))  

    # salvar toda a análise em um txt
    with open("data/analise_investimento_ap.txt", "w") as f:
        f.write("\n".join(output_lines))

    print("Arquivo txt salvo com sucesso na pasta 'data'")
    
    return resumo_bairros, resumo_percentual